In [2]:
from bs4 import BeautifulSoup
import time
import requests
import re
import pandas as pd


all_pmids = [] #This is the list where we will put all the PMIDs, loop them, and make them into URLs. 
out = []

#enter the start date range here - this will eventually go in the URL 
startdate = input('Enter the earliest date in m/d/yyyy format: ')

sm = startdate.split('/')[0]
sd = startdate.split('/')[1]
sy = startdate.split('/')[2]

sm = int(sm)
sd = int(sd)
sy = int(sy)

sm2 = startdate.split('/')[0]
sd2 = startdate.split('/')[1]
sy2 = startdate.split('/')[2]

sm2 = str(sm2)
sd2 = str(sd2)
sy2 = str(sy2)

if sm < 10 and '0' not in sm2:
    sm2 = '0' + sm2

if sd < 10 and '0' not in sd2:
    sd2 = '0' + sd2

#enter the end date range here this will eventually go in the URL 
enddate = input('Enter the latest date in m/d/yyyy format: ')

m = enddate.split('/')[0]
d = enddate.split('/')[1]
y = enddate.split('/')[2]

m = int(m)
d = int(d)
y = int(y)

m2 = enddate.split('/')[0]
d2 = enddate.split('/')[1]
y2 = enddate.split('/')[2]

m2 = str(m2)
d2 = str(d2)
y2 = str(y2)


if m < 10 and '0' not in m2:
    m2 = '0' + m2

if d < 10 and '0' not in d2:
    d2 = '0' + d2

Enter the earliest date in m/d/yyyy format:  12/10/2000
Enter the latest date in m/d/yyyy format:  12/30/2019


In [4]:
# Below is the text file with your terms.  If you have more than one word in a search term, you can either add a plus in the list itself or use the split command
# You want it to look like this: this+is+my+search+term
terms= open("lit_search_terms.txt", "r") 
terms = terms.read().split('\n')

for term in terms: #at the end of the url in full_url is a number.  This will indicate the number of results that can show up in a search.  The max number is 200. Right now, I have it at 20. 
    full_url = 'https://pubmed.ncbi.nlm.nih.gov/?term=%28'+term+'%5BText+Word%5D%29+AND+%28%28%22'+ sy2 +'%2F'+ sm2 + '%2F'+ sd2 +'%22%5BDate+-+Publication%5D+%3A+%22'+y2+'%2F'+m2+'%2F'+d2+'%22%5BDate+-+Publication%5D%29%29&sort=pubdate&size=20'                                                  
    response = requests.get(full_url)
    soup = BeautifulSoup(response.content, 'html.parser')
    print('Searching for: '+term)
    #When there's just one result, it automatically takes you to the one abstract the shows up.  Unfortunately this messes with the process of extracting all PMIDs and throwing them into a list.
    #When this happens, you'd just tell it to start grabbing the abstract information if the result is 1.  If it's more than 1, start grabbing the PMIDs per usual. 
    results = '1' if soup.find('span', {'class' : 'single-result-redirect-message'}) is not None else '>1'
 

    if results == '1': #Basically, we are saying that if there are no results, just look at the 1, get the PMID and throw it into list with other searches. 
        singlepmid = soup.find("meta",  property="og:url")
        singlepmid = str(singlepmid)
        singlepmid = singlepmid.split('<meta content="https://pubmed.ncbi.nlm.nih.gov/')[1]
        singlepmid = singlepmid.split('/')[0]
        all_pmids.append(singlepmid)
    else:
        singlepmid = None

    if results == '>1': #Most searches will have more than 1 result. In this case, it gets the PMIDs/URLs for each abstract and throws it into a list.
           
        pmids = soup.find_all('span', {'class' : 'docsum-pmid'})
        for p in pmids:
            p = p.get_text()
            all_pmids.append(p)

Searching for: telehealth+access
Searching for: access+to+care


In [5]:
    for pmid in all_pmids: #looping through the list of PMIDs
        url = 'https://pubmed.ncbi.nlm.nih.gov/'+pmid #each abstract URL is literally the same baseline URL with / and the PMID after
        response2 = requests.get(url)
        soup2 = BeautifulSoup(response2.content, 'html.parser')

        #Below we are getting the authors, journal, link to the full journal article, the abstract, and author affiliation
        #There are cleaner ways to do this, but for me it's faster to just find the data and parse the text around it.
        #If something is straightforward like a title or a table, you usually can just do soup.find('thehtmltag') and then .get_text() to get rid of the HTML tags

        #Authors
        authors = soup2.find(class_='authors-list').get_text(' ') 
        authors = authors.rstrip()
        authors = " ".join(authors.split())
        authors = authors.replace(' , ', ', ')

        #Journal
        journal = soup2.find(class_='journal-actions-trigger trigger') if soup2.find(class_='journal-actions-trigger trigger') is not None else ''
        journal = str(journal) if soup2.find(class_='journal-actions-trigger trigger') is not None else ''
        journal = journal.split('title="')[1] if soup2.find(class_='journal-actions-trigger trigger') is not None else ''
        journal = journal.split('">')[0] if soup2.find(class_='journal-actions-trigger trigger') is not None else ''
        journal = journal.title() if soup2.find(class_='journal-actions-trigger trigger') is not None else ''

        #Link to full article

        fulltext = soup2.find(class_='full-text-links-list') if soup2.find(class_='full-text-links-list') is not None else ''
        fulltext = str(fulltext) if soup2.find(class_='full-text-links-list') is not None else ''
        fulltext = fulltext.split('href="')[1] if soup2.find(class_='full-text-links-list') is not None else ''
        fulltext = fulltext.split('"')[0] if soup2.find(class_='full-text-links-list') is not None else ''

        # Abstract
        abstract = soup2.find(class_='abstract-content selected')
        abstract = str(abstract)
        abstract = abstract.rstrip()
        abstract = abstract.replace('<p>', '@p@')
        clean = re.compile('<.*?>')
        abstract = re.sub(clean, '', abstract)
        abstract = re.sub(r'\n\s*\n', '\n\n', abstract)
        abstract = " ".join(abstract.split())
        abstract = abstract.replace('@p@ ','\n'+'\n')
        abstract = abstract.lstrip()
        abstract = abstract.rstrip()

        # Affiliation
        affiliation = soup2.find(class_='item-list').get_text(' ') if soup2.find(class_='item-list') is not None else ''
        affiliation = str(affiliation)if soup2.find(class_='item-list')is not None else ''
        affiliation = affiliation.split('\n') if soup2.find(class_='item-list') is not None else ''
        affiliation = str(affiliation)  if soup2.find(class_='item-list') is not None else ''
        affiliation = affiliation.replace(" ', '']","").replace("['', ' ","").replace("', '",",") if soup2.find(class_='item-list')is not None else ''
    
        title = soup2.select('h1.heading-title')[0].text.strip()  

        data = {'title': title, 'authors': authors, 'affiliations': affiliation, 'abstract': abstract, 'journal': journal,'full_article_link': fulltext, 'url':url}
        time.sleep(5) #the robots.txt file on PubMed states that they want 5 seconds between requests. Change this as needed. 

        out.append(data) #We're throwing all the data into a new list and creating a dataframe

In [6]:
df = pd.DataFrame(out) #This is the dataframe
df.drop_duplicates() # This drops any duplicates.
df.to_excel('HPV.xlsx') #See the Pandas module for this.  If you wanted this to be a csv file, you'd just change it to df.to_csv('nameofyouroutput.csv). 
    